# Quora question pairs: data preparation

## Import packages

In [1]:
from __future__ import print_function

import numpy as np
import csv, json
from zipfile import ZipFile
from os.path import expanduser, exists

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.data_utils import get_file



import sys
import os 
import pandas as pd
from tqdm import tqdm


import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline



import spacy

# import utils 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from __future__ import absolute_import
from __future__ import print_function
from IPython.lib.display import FileLink


import pickle 

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5103)


## Initialize global variables

In [34]:
path = '/home/ubuntu/quora/'
data_home = path +"data/"


KERAS_DATASETS_DIR = data_home+"cache/"

# KERAS_DATASETS_DIR = expanduser('~/.keras/datasets/')
QUESTION_PAIRS_FILE_URL = 'http://qim.ec.quoracdn.net/quora_duplicate_questions.tsv'
QUESTION_PAIRS_FILE = 'quora_duplicate_questions.tsv'
GLOVE_ZIP_FILE_URL = 'http://nlp.stanford.edu/data/glove.840B.300d.zip'
GLOVE_ZIP_FILE = 'glove.840B.300d.zip'
GLOVE_FILE = 'glove.840B.300d.txt'
Q1_TRAINING_DATA_FILE = 'q1_train.npy'
Q2_TRAINING_DATA_FILE = 'q2_train.npy'
Q1_TESTING_DATA_FILE = 'q1_test.npy'
Q2_TESTING_DATA_FILE = 'q2_test.npy'


LABEL_TRAINING_DATA_FILE = 'label_train.npy'
WORD_EMBEDDING_MATRIX_FILE = 'word_embedding_matrix.npy'
NB_WORDS_DATA_FILE = 'nb_words.json'
MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 35#25
EMBEDDING_DIM = 300

## Download and extract questions pairs data

In [3]:
# df_train = pd.read_csv(data_home+'train.csv' , encoding='utf-8'   )
# df_test = pd.read_csv(data_home+'test.csv' , encoding='utf-8'   )

############ read cleaned data ##########
df_train = pd.read_csv(data_home+'train_clean_vB.csv' , encoding='utf-8'   )
df_test = pd.read_csv(data_home+'test_clean_vB.csv' , encoding='utf-8'   )


In [4]:
df_test.isnull().sum()

test_id        0
q1_clean_vB    0
q2_clean_vB    0
dtype: int64

In [5]:
df_train = df_train.fillna("empty")
df_test = df_test.fillna("empty")

In [6]:
df_train.head(2)


,id,qid1,qid2,is_duplicate,q1_clean_vB,q2_clean_vB
0,0,1,2,0,What is step by step guide to invest in share ...,What is step by step guide to invest in share ...
1,1,3,4,0,What is story of Kohinoor Koh - i - Noor Diamond?,What would happen if Indian government stole K...


In [8]:
#getting rid of original question1 and question2 columns for simplicity

df_train['question1']  =  df_train.q1_clean_vB
df_train['question2'] =  df_train.q2_clean_vB

df_test['question1'] =  df_test.q1_clean_vB
df_test['question2'] =  df_test.q2_clean_vB

In [9]:
df_train = df_train.drop(["q1_clean_vB","q2_clean_vB"],axis=1)
df_test = df_test.drop(["q1_clean_vB","q2_clean_vB"],axis=1)

In [9]:

# df_train['question1'] = df_train['question1'].apply(lambda x: x.encode('utf-8'))
# df_train['question2'] = df_train['question2'].apply(lambda x: x.encode('utf-8'))

# df_test['question1'] = df_test['question1'].apply(lambda x: x.encode('utf-8'))
# df_test['question2'] = df_test['question2'].apply(lambda x: x.encode('utf-8'))

In [9]:
df_train.head()

,id,qid1,qid2,is_duplicate,question1,question2
0,0,1,2,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,1,3,4,0,What is the story of Kohinoor (Koh i Noor) Dia...,What would happen if the Indian government sto...
2,2,5,6,0,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,3,7,8,0,Why am I mentally very lonely? How can I solve...,find the remainder when [math]23^{24}[/math] i...
4,4,9,10,0,"Which one dissolve in water quickly sugar, sal...",Which fish would survive in salt water?


In [10]:
question1_train = df_train.question1.astype(str)
question2_train = df_train.question2.astype(str)
is_duplicate = df_train.is_duplicate

In [11]:
question1_test = df_test.question1.astype(str)
question2_test = df_test.question2.astype(str)

In [12]:

print('Question pairs: training: %d test:' % len(question1_train)  , len(question1_test))

Question pairs: training: 404290 test: 2345796


## Build tokenized word index

In [13]:
#use test questions also to create tokens 

train_questions = question1_train + " "+ question2_train
test_questions = question1_test + " "+ question2_test

len(test_questions)

2345796

In [14]:
questions = train_questions.append( test_questions)

In [15]:
test_questions[2345795]

'What is best medication equation erectile dysfunction? How do I out get rid of Erectile Dysfunction?'

In [19]:
%%time

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(questions)



CPU times: user 50.8 s, sys: 20 ms, total: 50.8 s
Wall time: 50.8 s


In [18]:
#creating reverse index to word map (for paraphrase generation (NOT DONE))
# inv_map = {v: k for k, v in tokenizer.word_index.iteritems()}
# with open(data_home+"cache/tokenizer_reverse_word_index.dat",'wb') as handle:
#     pickle.dump(inv_map,handle)

In [36]:
%%time

question1_word_sequences = tokenizer.texts_to_sequences(question1_train)
question2_word_sequences = tokenizer.texts_to_sequences(question2_train)
word_index = tokenizer.word_index

print("Words in index: %d" % len(word_index))

Words in index: 120773
CPU times: user 6.1 s, sys: 20 ms, total: 6.12 s
Wall time: 6.11 s


In [37]:
question2_word_sequences[0]

[1, 2, 1252, 60, 1252, 2919, 7, 577, 6, 757, 365]

In [22]:
%%time
testq1_word_sequences = tokenizer.texts_to_sequences(question1_test)
testq2_word_sequences = tokenizer.texts_to_sequences(question2_test)

CPU times: user 40.2 s, sys: 404 ms, total: 40.6 s
Wall time: 40.6 s


## Download and process GloVe embeddings

In [17]:

# if not exists(KERAS_DATASETS_DIR + GLOVE_ZIP_FILE):
#     zipfile = ZipFile(get_file(GLOVE_ZIP_FILE, GLOVE_ZIP_FILE_URL))
#     zipfile.extract(GLOVE_FILE, path=KERAS_DATASETS_DIR)
    


this is how a word vector looks like:

['melodies', '0.39203', '0.76391', '-0.27438', '0.20478', '0.28002', '0.15006', '0.27144', '0.03625', '-0.005431', '0.79186', '0.97261', '0.32796', '-0.16917', '0.37354', '0.35588', '0.05971', '0.066368', '0.39296', '-0.034228', '-0.16004', '-0.3086', '0.61723', '0.13225', '-0.30863', '-0.16481', '0.18971', '0.36708', '0.4132', '0.8603', '-0.10542', '0.28735', '-0.43696', '-0.64395', '-0.16088', '0.32564', '-0.030691', '-0.64959', '0.28413', '0.42177', '0.12257', '-0.11246', '-0.084039', '-0.13831', '0.42894', '0.59371', ...]

In [23]:
%%time

# print("Processing", GLOVE_FILE)
# embeddings_index = {}

# with open(KERAS_DATASETS_DIR + GLOVE_FILE) as f: #, encoding='utf-8'
#     for line in f:
#         values = line.split(' ')
#         word = values[0]
#         embedding = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = embedding




Processing glove.840B.300d.txt
CPU times: user 2min 11s, sys: 3.08 s, total: 2min 14s
Wall time: 2min 15s


In [2]:
%load_ext Cython


In [23]:

import pickle


with open(data_home+"cache/embeddings_index.npy", 'rb') as handle:
    embeddings_index = pickle.load(handle)


In [24]:
print('Word embeddings: %d' % len(embeddings_index))
# embeddings_index["fuss"]

Word embeddings: 2196016


In [24]:
# np.save(open(data_home+"cache/embeddings_index.npy", 'wb'), embeddings_index)
# embeddings_index = np.load(open(emb_index_path, 'rb'))

# with open(data_home+"cache/embeddings_index.npy", 'wb') as handle:
#     pickle.dump(embeddings_index, handle, protocol=pickle.HIGHEST_PROTOCOL)



# print a == b    

## Prepare word embedding matrix

Null word embeddings: 65893 (v1 cleaning)
Null word embeddings: 32955 (v2 cleaning) halved!

In [25]:
min(MAX_NB_WORDS, len(word_index))

120773

In [38]:
%%time

nb_words = min(MAX_NB_WORDS, len(word_index))
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
no_embeddings = []


for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector
    else:
        no_embeddings.append(word)
        word_embedding_matrix[i] = np.random.rand(300)



CPU times: user 424 ms, sys: 60 ms, total: 484 ms
Wall time: 481 ms


In [39]:
print('Null word embeddings: %d' % len(no_embeddings))#np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

Null word embeddings: 33420


In [40]:
no_embeddings

['murwara',
 'daiict',
 'utnapishtim',
 'devonians',
 'fawx',
 'paiza',
 'turram',
 'lottoland',
 'chanthaburi',
 'googlenet',
 'zandikhohlisa',
 'gazatted',
 'freesteamgifts',
 'naturopathix',
 'throwaround',
 'vologda',
 'ctk3200',
 'girokonto',
 'targu',
 'starboy',
 'krampusnacht',
 'naache',
 '2300mhz',
 'movietv',
 'trivikram',
 '27i',
 'bhubaneshwar',
 'oct14',
 't250si',
 't250sl',
 'imporatnace',
 'istributed',
 'diomand',
 'mclaurin',
 'yaghan',
 'ohhio',
 'issual',
 'thamirabarani',
 'jspatch',
 'pgdhrm',
 'ronalds',
 'unlearnable',
 'twenty19',
 'kayasthas',
 'vasuda',
 'inorgnc',
 'avma',
 'iraiva',
 'stavara',
 'borstar',
 '2phz',
 'sy0',
 '900eur',
 'torqouise',
 'oliveboard',
 'chandrabati',
 'pinth',
 '28teeth',
 '180deggrid',
 'phoritto',
 'barbro',
 'censys',
 'oritani',
 'regstn',
 'catracha',
 'dlvrit',
 'p207tx',
 'uov',
 'cryolife',
 'sizr',
 'vrushali',
 'shinawatra',
 'buuz',
 'class11th',
 '9w1',
 'pandodaily',
 'moonmeander',
 '9w8',
 'vilom',
 'raahaten',
 '

In [41]:
np.save(open(data_home+"cache/no_embeddings_B.npy", 'wb'), no_embeddings)


## Prepare training data tensors

In [43]:
q1_train_data = pad_sequences(question1_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
q2_train_data = pad_sequences(question2_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)

q1_test_data =  pad_sequences(testq1_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
q2_test_data = pad_sequences(testq2_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)


labels = np.array(is_duplicate, dtype=int)
print('Shape of question1 data (training) tensor:', q1_train_data.shape)
print('Shape of question2 data (training) tensor:', q2_train_data.shape)
print('Shape of label tensor:', labels.shape)

Shape of question1 data (training) tensor: (404290, 35)
Shape of question2 data (training) tensor: (404290, 35)
Shape of label tensor: (404290,)


## Persist training and configuration data to files

In [44]:
np.save(open(data_home+"cache/"+Q1_TRAINING_DATA_FILE, 'wb'), q1_train_data)
np.save(open(data_home+"cache/"+Q2_TRAINING_DATA_FILE, 'wb'), q2_train_data)
np.save(open(data_home+"cache/"+LABEL_TRAINING_DATA_FILE, 'wb'), labels)
np.save(open(data_home+"cache/"+WORD_EMBEDDING_MATRIX_FILE, 'wb'), word_embedding_matrix)

with open(data_home+"cache/"+NB_WORDS_DATA_FILE, 'w') as f:
    json.dump({'nb_words': nb_words}, f)

In [45]:
np.save(open(data_home+"cache/"+Q1_TESTING_DATA_FILE, 'wb'), q1_test_data)
np.save(open(data_home+"cache/"+Q2_TESTING_DATA_FILE, 'wb'), q2_test_data)